# Preprocessing


## 1. Import libraries and read data

In [21]:
import pandas as pd

In [22]:
# Đọc dữ liệu
train = pd.read_csv("house_prices/train.csv")
test = pd.read_csv("house_prices/test.csv")

## 2. Execute missing data (preprocessing)

### 2.1. Check missing data 

In [23]:
print(f"Total original missing data in training set: {train.isnull().sum().sum()}")
print(f"Total original missing data in test set: {test.isnull().sum().sum()}")

Total original missing data in training set: 7829
Total original missing data in test set: 7878


In [24]:
missing = train.isnull().sum()
print(missing)

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [25]:
print('Missing data trong training set:')
missing[missing > 0].sort_values(ascending=False)

Missing data trong training set:


PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
dtype: int64

In [26]:
print('Missing data trong test set:')
missing_test = test.isnull().sum()
missing_test[missing_test > 0].sort_values(ascending=False)

Missing data trong test set:


PoolQC          1456
MiscFeature     1408
Alley           1352
Fence           1169
MasVnrType       894
FireplaceQu      730
LotFrontage      227
GarageQual        78
GarageCond        78
GarageYrBlt       78
GarageFinish      78
GarageType        76
BsmtCond          45
BsmtQual          44
BsmtExposure      44
BsmtFinType1      42
BsmtFinType2      42
MasVnrArea        15
MSZoning           4
Functional         2
BsmtFullBath       2
Utilities          2
BsmtHalfBath       2
Exterior1st        1
Exterior2nd        1
TotalBsmtSF        1
BsmtUnfSF          1
BsmtFinSF2         1
BsmtFinSF1         1
KitchenQual        1
GarageArea         1
GarageCars         1
SaleType           1
dtype: int64

### 2.2. Categorize missing data
Divide missing data into 4 main categories:
- Group 1: Columns where "NA" means "not present" (i.e., None).
- Group 2: Numeric columns where NA also means "not present".
- Group 3: Columns that require estimation based on other data.
- Group 4: Columns with few missing values → fill with the most common value (mode).

General principles for filling in missing values:

1. For object-type columns (e.g., GarageType, FireplaceQu, BsmtCond):
- If missing means the feature doesn't exist (e.g., no basement → missing BsmtCond), then fill with 'None'.
- If missing is likely due to data entry error but can be inferred → fill with mode().

2. For integer-type variables that represent counts (e.g., BsmtFullBath, GarageCars):
- If missing due to the absence of a basement or garage → fill with 0.
- If the reason is unclear → inspect manually (e.g., visualize with plots, check if all related basement columns are missing for the same row).

3. For float-type variables like area (e.g., LotFrontage, GarageArea, MasVnrArea):
- Usually missing due to measurement error or omission.
- Fill with median() to avoid the influence of outliers (since median is more robust than mean).

4. For Year, Date, or other special types (e.g., GarageYrBlt):
- If missing because there's no garage → choose a consistent strategy: fill with 0, or with YearBuilt, or with 'None', depending on the intended analysis.
- Use logic and domain knowledge to guide your decision.



#### Group 1: Categorical columns where NA means "not present" (e.g., no pool).

In [27]:
# Xử lý: điền "None"
cols_none = [
    'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtCond', 'BsmtFinType1',
    'MasVnrType', 'MSZoning', 'Utilities', 'Functional', 'Exterior1st', 
    'Exterior2nd', 'SaleType', 'KitchenQual', 'Electrical'
]

for col in cols_none:
    train[col] = train[col].fillna("None")
    
for col in cols_none:
    test[col] = test[col].fillna("None")

#### Group 2: Numeric columns where NA means "not present"

In [28]:
# Xử lý: điền 0
cols_fill_zero = ['MasVnrArea', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 
                       'BsmtUnfSF', 'TotalBsmtSF', 'GarageCars', 'GarageArea']

train[cols_fill_zero] = train[cols_fill_zero].fillna(0)
test[cols_fill_zero] = test[cols_fill_zero].fillna(0)

#### Group 3: Columns that require imputation based on other data

In [29]:
# LotFrontage – chiều rộng mặt tiền → bạn có thể điền bằng trung bình theo khu phố (Neighborhood)
train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)
# 2 cách của train và test như nhau
test["LotFrontage"] = test["LotFrontage"].fillna(test["LotFrontage"].median())

#### Group 4: Columns with few missing values → fill with the mode (most frequent value)

In [30]:
# .mode() trả về danh sách các giá trị phổ biến nhất. (sắp xếp từ nhiều xuống ít)
# .mode()[0] chọn giá trị đầu tiên trong số đó (để dùng cho việc điền missing value).

for col in ["MSZoning", "Utilities", "Functional", "Exterior1st", "Exterior2nd", 
            "KitchenQual", "SaleType", "Electrical"]:
    test[col] = test[col].fillna(test[col].mode()[0])

### 2.3. Checking for missing data 

+ All missing data has been properly handled (better than dropping it directly).
+ Rerun train.isnull().sum().sum() to confirm that all missing data has been removed.

In [31]:
print(f"Total current missing data in training set: {train.isnull().sum().sum()}")
print(f"Total current missing data in test set: {test.isnull().sum().sum()}")

Total current missing data in training set: 0
Total current missing data in test set: 0


In [32]:
train.isnull().sum()[train.isnull().sum() > 0]

Series([], dtype: int64)

In [33]:
test.isnull().sum()[test.isnull().sum() > 0]

Series([], dtype: int64)

## 3. Convert Categorical (Text) Variable to Numeric Variable (Encoding)

Some machine learning algorithms like Linear Regression cannot work directly with text (categorical) data. Therefore:

+ Label Encoding: used when the variable is ordinal (has a meaningful order)

+ One-Hot Encoding: used when the variable is nominal (no inherent order)

Use pd.get_dummies() to perform One-Hot Encoding for both train and test sets:

Using pd.get_dummies() will:

1. dentify all columns in the DataFrame that contain data of type object or category (i.e., text).
2. For each of those columns, create a new column for every unique value in the original column.
3. In these new columns, the values are:
+ 1 if the row has the corresponding value
+ 0 if it does not

In [34]:
# Dùng pd.get_dummies() để One-Hot Encoding cho cả train và test
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [35]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,False,False,False,True,False,False,False,False,True,False
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,False,False,False,True,False,False,False,False,True,False
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,False,False,False,True,False,False,False,False,True,False
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,False,False,False,True,True,False,False,False,False,False
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,False,False,False,True,False,False,False,False,True,False


But sometimes train and test have different number of columns after one-hot. To handle:

In [36]:
# Đảm bảo train và test có cùng số lượng và thứ tự cột (sau khi get_dummies()), để mô hình có thể học và dự đoán chính xác, không bị lỗi do lệch cột.

train, test = train.align(test, join='left', axis=1, fill_value=0)

# Sử dụng train.align() – đây là một hàm trong Pandas trả về 2 DataFrame, nên ta phải gán cả hai đầu ra cùng lúc.
# Cụ thể: train.align(test) trả về 2 giá trị:
# ~ Một bản train đã được căn chỉnh lại cột theo thứ tự của test, nhưng chỉ giữ những cột có ở train
# ~ Một bản test đã được căn chỉnh lại theo train

# join='left'	Dùng các cột của train làm chuẩn. Các cột có trong train mà không có trong test sẽ được thêm vào test và điền bằng fill_value. 
#               Các cột chỉ có trong test nhưng không có trong train sẽ bị loại bỏ.
# axis=1	    Ghép theo cột (columns).
# fill_value=0	Nếu test thiếu cột nào so với train, thì cột đó sẽ được tạo và gán giá trị mặc định là 0.

## 4. Split to features and target (X và y)

Prepare data to train the model

### 4.1. Create feature set X (input)

X = train.drop("SalePrice", axis=1).

Meaning:
+ X is the input set (features/input data for the model to learn).
+ You are dropping the SalePrice column from the train DataFrame, meaning:
+ The input data is all columns except SalePrice
+ Because SalePrice is the output (label/target) — house price — what you want the model to predict

→ X = feature set.

In [37]:
X_full_train = train.drop('SalePrice', axis=1)
# axis=1 → thao tác theo chiều cột (column)

X_test = test.drop('SalePrice', axis=1)

### 4.2. Create set of true house price y (output)

y = train["SalePrice"]

Meaning:
+ y is the output/label, which is the actual house price, which you want the model to learn to predict.

In [38]:
y_full_train = train['SalePrice']
y_test = test['SalePrice']

## 5. Feature Scaling

Many models need to scale data to the same unit (e.g. KNN, SVM, Ridge, Lasso, etc.)

Use StandardScaler (z-score normalization) (because the model uses Linear Regression to predict)

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_full_train = scaler.fit_transform(X_full_train)
# fit_transform(X) - Làm 2 việc:
# ~ fit(X): Tính toán mean và standard deviation trên X, sau đó σ và μ (của X) được lưu lại trong scaler.
# ~ transform(X): Áp dụng công thức chuẩn hóa (z-score) 

X_test = scaler.transform(X_test)  # Áp dụng chuẩn hóa của X (cái cột X_train và X_test phải giống nhau)

## 6. Train-Test Split (to evaluate model)

From the initial training set, we will split it into 2 parts:
+ Train (take 0.8 - to train the model)
+ Cross-validation (take 0.2 - to evaluate the model)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_full_train, y_full_train, test_size=0.2, random_state=42)